In [1]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms as T

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"